In [1]:
import numpy as np
from tqdm import tqdm
from copy import copy

In [2]:
def parse_input(input_file: str) -> np.array:
    with open(input_file, "r") as file:
        grid_array = None

        for line in file:
            line_chars = list(line.strip()) 
            #print(f"Line: {line_chars}, lenght: {len(line_chars)}")

            line_array = np.array(line_chars)

            # first line: assignment
            if grid_array is None:
                grid_array = line_array
            # 2nd to nth line: append to existing 2d-array
            else:
                grid_array = np.vstack((grid_array, line_array))

    return grid_array


def find_empty_index(rock_array: np.array, row_index: int, col_index: int) -> int:
    for row in range(row_index-1, -1, -1):
        if rock_array[row, col_index] == 'O' or rock_array[row, col_index] == '#':
            return row+1
    return 0


def sort_rock_string(input_str: str) -> str:
    parts = input_str.split('#')
    sorted_parts = [''.join(sorted(part, key=lambda x: x!= 'O')) for part in parts]
    return '#'.join(sorted_parts)

def roll_direction(rock_array: np.array, direction: str = "north") -> np.array:
    if direction in ['north', 'south']:
        for col in range(rock_array.shape[1]):
            if direction == 'north':
                column = "".join(rock_array[:, col])
                fixed_column = sort_rock_string(column)
                rock_array[:, col] = list(fixed_column)
            elif direction == 'south':
                column = "".join(rock_array[:, col][::-1])
                fixed_column = sort_rock_string(column)
                rock_array[:, col] = list(fixed_column)[::-1]
    if direction in ['east', 'west']:
        for row_index in range(rock_array.shape[0]):
            if direction == 'east':
                row = "".join(rock_array[row_index, :][::-1])
                fixed_row = sort_rock_string(row)
                rock_array[row_index, :] = list(fixed_row)[::-1]
            elif direction == 'west':
                row = "".join(rock_array[row_index, :])
                fixed_row = sort_rock_string(row)
                rock_array[row_index, :] = list(fixed_row)
    return rock_array

def calculate_north_load(rock_array: np.array) -> int:
    coordinates = np.where(rock_array == 'O')
    total_load = 0
    for row in coordinates[0]:
        total_load += abs(row - rock_array.shape[0])
    return total_load

In [3]:
def solve_part1(input_file: str) -> int:
    rock_array = parse_input(input_file)
    #print(rock_array)
    rock_array = roll_direction(rock_array, "north")
    #print(rock_array)
    total_load = calculate_north_load(rock_array)
    return total_load

In [127]:
solve_part1('example.txt')

136

In [124]:
def solve_part2(input_file: str) -> int:
    rock_array = parse_input(input_file)
    target_cycles = 1000000000
    past_rock_arrays = [copy(rock_array)]
    values = [calculate_north_load(rock_array)]
    for current_cycle in range(1, target_cycles+1):
        rock_array = roll_direction(rock_array, "north")
        rock_array = roll_direction(rock_array, "west")
        rock_array = roll_direction(rock_array, "south")
        rock_array = roll_direction(rock_array, "east")
        values.append(calculate_north_load(rock_array))
        for index, past_array in enumerate(past_rock_arrays):
            if np.array_equal(rock_array, past_array):
                cycle_start = index 
                cycle_length = current_cycle - cycle_start
                print(f"cycle start: index {cycle_start}, cycle length: {cycle_length}")
                print(f"found repeat at cycle {current_cycle}, target repeats from {((target_cycles-cycle_start) % cycle_length) + cycle_start}")
                solution_array = past_rock_arrays[((target_cycles-cycle_start) % cycle_length) + cycle_start]
                return calculate_north_load(solution_array)
        past_rock_arrays.append(copy(rock_array))
    total_load = calculate_north_load(rock_array)
    return total_load

In [126]:
solve_part2('example.txt')

cycle start: index 3, cycle length: 7
found repeat at cycle 10, target repeats from 6


64